In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tcn import tcn, tcn_full_summary
import DataFunctions as data

2024-05-15 16:11:36.738012: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 16:11:36.793766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 16:11:36.793801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 16:11:36.795675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 16:11:36.805643: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Se obtienen los datos de los sensores y las etiquetas deducidas
print("Obteniendo datos...")
data_2206_17_bag, tags_220617_bag = data.get_data('/home/cmonedero/TFG/User1/Motion/User1_220617_Bag.csv', '/home/cmonedero/TFG/User1/220617/Label_New_User1_220617.txt')

Obteniendo datos...


Hecho!



In [3]:
x_windows_220617_bag, y_windows_220617_bag = data.divide_data_arrays_into_windows(data_2206_17_bag, tags_220617_bag, moda=True, window_size=500)
print("Tamaño ventanas X --> ", x_windows_220617_bag.shape)
print("Tamaño ventanas Y --> ", y_windows_220617_bag.shape)

Hecho!

Tamaño ventanas X -->  (3651, 500, 6)
Tamaño ventanas Y -->  (3651,)


In [4]:
x_train_220617_bag, y_train_220617_bag, x_test_220617_bag, y_test_220617_bag = data.divide_train_test(x_windows_220617_bag, y_windows_220617_bag, 0.7)
print(f'Shape x_train --> {x_train_220617_bag.shape}')
print(f'Shape y_train --> {y_train_220617_bag.shape}')
print(f'Shape x_test --> {x_test_220617_bag.shape}')
print(f'Shape y_train --> {y_test_220617_bag.shape}')

Shape x_train --> (2555, 500, 6)
Shape y_train --> (2555,)
Shape x_test --> (1096, 500, 6)
Shape y_train --> (1096,)


In [5]:
print(np.unique(y_train_220617_bag))

[1 2 5 6]


In [6]:
inputs = Input(shape=(500,6))
x = inputs
x = tcn.TCN(input_shape=(500,6),
            use_skip_connections=False,
            use_batch_norm=False,
            use_weight_norm=False,
            use_layer_norm=False,
            return_sequences=True,
            nb_filters=32)(x)
print(f'Shape del input despues de TCN {x.shape}')
x = Flatten()(x)
print(f'Shape después Flatten {x.shape}')
outputs = Dense(8, activation='softmax')(x)
print(f'Shape de la capa final {outputs.shape}')

modelo = keras.Model(inputs, outputs)

# modelo = Sequential()
# modelo.add(Dense(100, activation='relu'))
# modelo.add(Dense(units=8))
modelo.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

2024-05-15 16:11:54.602653: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 970 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6
2024-05-15 16:11:54.605126: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 40933 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:98:00.0, compute capability: 8.6


Shape del input despues de TCN (None, 500, 32)
Shape después Flatten (None, 16000)
Shape de la capa final (None, 8)


In [8]:
modelo.fit(tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float32), epochs=10)

Epoch 1/10


2024-05-15 16:12:09.662314: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-05-15 16:12:09.790676: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-15 16:12:10.066376: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-15 16:12:10.213852: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-15 16:12:10.213916: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance 

80/80 [==============================] - 9s 31ms/step - loss: 416.4726 - accuracy: 0.5898
Epoch 2/10
80/80 [==============================] - 2s 20ms/step - loss: 4.3521 - accuracy: 0.8204
Epoch 3/10
80/80 [==============================] - 2s 20ms/step - loss: 2.3238 - accuracy: 0.8701
Epoch 4/10
80/80 [==============================] - 2s 20ms/step - loss: 4.6387 - accuracy: 0.8333
Epoch 5/10
80/80 [==============================] - 2s 20ms/step - loss: 3.9614 - accuracy: 0.8622
Epoch 6/10
80/80 [==============================] - 2s 20ms/step - loss: 2.7810 - accuracy: 0.8814
Epoch 7/10
80/80 [==============================] - 2s 20ms/step - loss: 1.4665 - accuracy: 0.9123
Epoch 8/10
80/80 [==============================] - 2s 20ms/step - loss: 3.2472 - accuracy: 0.8771
Epoch 9/10
80/80 [==============================] - 2s 20ms/step - loss: 3.4313 - accuracy: 0.8877
Epoch 10/10
80/80 [==============================] - 2s 20ms/step - loss: 2.8595 - accuracy: 0.9041


In [9]:
evals = modelo.evaluate(tf.constant(x_test_220617_bag), tf.cast(tf.constant(y_test_220617_bag), tf.float32))

35/35 [==============================] - 1s 18ms/step - loss: 341.5082 - accuracy: 0.7026


In [10]:
modelo.save('TCNMemo/tcn.h5')

/home/cmonedero/TFG/tfg_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
modelo_tonto_json = modelo_tonto.to_json()
with open('TCNMemo/model.json', "w") as json_file:
    json_file.write(modelo_tonto_json)
modelo.save_weights('TCNMemo/weights.h5')